In [1]:
# Generate data for the Darcy flow problem
# Input: a - the permeability "ground truth"
# output: u - the flow through the permeability
# data generated:
# 2 datasets of 200 'Devito' pairs, to compare with the ground truth, with resolutions of:
# 128, 64

In [ ]:
pip install torch

In [ ]:
pip install tensorflow

In [1]:
import torch
import math
from devito import div, grad, Eq, Operator, TimeFunction, Function, solve, Grid, configuration
import numpy as np
from random_fields import GaussianRF
from timeit import default_timer
import scipy.io

2022-08-29 17:21:20.688714: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-29 17:21:20.688775: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import os

In [3]:
os.environ['DEVITO_LANGUAGE'] = 'openmp'

os.environ['OMP_SCHEDULE'] = 'STATIC'

os.environ['OMP_PROC_BIND'] = 'true'

os.environ['OMP_PLACES'] = 'cores'

In [4]:

'''
function to generate 'u' from 'a' using Devito
parameters 
-----------------
perm: tensor of size (1, s, s)
    this is "a"
f: tensor of size (s, s)
    the forcing function f(x) = 1
s: int
    the resolution
 '''
def darcy_flow_2d(perm, f, s):
    
    # a(x) is the coefficients
    # f is the forcing function
    # give the a and f1 functions the same data as inputs perm and f
    for i in range(s):
        f1.data[i] = f[i]
        a.data[i] = perm[i]
    
    # call operator for the nth psuedo-timestep
    if s == 128:
        n = 7500
    else:
        n = 3750
    
    op(time=n)
   
    return torch. from_numpy(u.data[0])

In [5]:
# Silence the runtime performance logging
configuration['log-level'] = 'ERROR'

# this is used to generate "devito" data results, to compare to the FNO results and the ground truth

# Use the inputs from the ground truth to generate new outputs to compare
test_128 = scipy.io.loadmat('df_test_128.mat') 
test_64 = scipy.io.loadmat('df_test_64.mat') 

# for each test mat file:
for testcase in [test_128, test_64]:
   
    # number of grid points on [0,1]^2 
    s = testcase['coeff'].shape[1]
    print("this is s", s)
    
    # Number of solutions to generate
    N = 200

    # Forcing function, f(x) = 1 
    f = np.ones((s, s))

    # create s x s grid with spacing 1
    grid = Grid(shape=(s, s), extent=(1.0,1.0))

    # create function on grid
    # space order of 2 to enable 2nd derivative
    # TimeFunction for u can be used despite the lack of a time-dependence. This is done for psuedotime
    u = TimeFunction(name='u', grid=grid, space_order=2)
    a = Function(name='a', grid=grid, space_order=2)
    f1 = Function(name='f1', grid=grid, space_order=2)

    # define 2D Darcy flow equation
    equation_u = Eq(-a*u.laplace,f1)

    # Let SymPy solve for the central stencil point
    stencil = solve(equation_u, u)

    # Let our stencil populate the buffer `u.forward`
    update = Eq(u.forward, stencil)

    # Boundary Conds
    x, y = grid.dimensions
    t = grid.stepping_dim
    nx = s
    ny = s
    bc = [Eq(u[t+1, 0, y],u[t+1, 1,y])]  # du/dx = 0 for x=0.
    bc += [Eq(u[t+1,nx-1, y],u[t+1,nx-2, y])]  # du/dx = 0 for x=1.
    bc += [Eq(u[t+1, x, 0],u[t+1,x ,1])]  # du/dx = 0 at y=0
    bc += [Eq(u[t+1, x, ny-1],u[t+1, x, ny-2])]  # du/dx=0 for y=1
    # u=0 for all sides
    bc = [Eq(u[t+1, x, 0], 0.)]
    bc += [Eq(u[t+1, x, ny-1], 0.)]
    bc += [Eq(u[t+1, 0, y], 0.)]
    bc += [Eq(u[t+1, nx-1, y], 0.)]

    op = Operator([update] + bc)


    # all the inputs
    coeff = np.zeros(shape=(N, s, s))
    #all the solutions
    sol = torch.zeros(N, s, s)
     
    c = 0
    t0 = default_timer()
    
    # for each input generate a new solution
    for each_input in testcase['coeff']:
        
        w1 = each_input
        
        #Solve df
        output = darcy_flow_2d(w1, f, s)
        
        coeff[c:(c+1),...] = w1
        sol[c:(c+1),...] = output
        
        c += 1
        t1 = default_timer()
        print("sample no. :", c, "Time taken:", t1-t0)
    
    scipy.io.savemat('df_devito_{}.mat'.format(s), mdict={'coeff': coeff, 'sol': sol.cpu().numpy()})

this is s 128
sample no. : 1 Time taken: 0.40384356025606394
sample no. : 2 Time taken: 0.5324056725949049
sample no. : 3 Time taken: 0.651508585549891
sample no. : 4 Time taken: 0.7701763212680817
sample no. : 5 Time taken: 0.8913664072751999
sample no. : 6 Time taken: 1.0228362949565053
sample no. : 7 Time taken: 1.150304819457233
sample no. : 8 Time taken: 1.2821287848055363
sample no. : 9 Time taken: 1.402497736737132
sample no. : 10 Time taken: 1.5271877311170101
sample no. : 11 Time taken: 1.646625217050314
sample no. : 12 Time taken: 1.7668254980817437
sample no. : 13 Time taken: 1.8860299428924918
sample no. : 14 Time taken: 2.007365363650024
sample no. : 15 Time taken: 2.1288217278197408
sample no. : 16 Time taken: 2.2501185182482004
sample no. : 17 Time taken: 2.368564614094794
sample no. : 18 Time taken: 2.502957816235721
sample no. : 19 Time taken: 2.622951534576714
sample no. : 20 Time taken: 2.74183935765177
sample no. : 21 Time taken: 2.860814212821424
sample no. : 22 Ti